In [ ]:
# Install dependencies
!pip install transformers
!pip install torch

In [ ]:
# Import dependencies
import torch
from transformers import BertTokenizer, BertModel
from transformers import pipeline

In [ ]:
# Check if a GPU is available
torch.cuda.is_available()

saving progress from original notebook here if necessary

In [ ]:
# Create a pipeline to analyze sentiment of player bios with a max length of 512 tokens
nlp = pipeline('sentiment-analysis',
               model='siebert/sentiment-roberta-large-english', max_length=512)

# Create a new column in the DataFrame to hold the sentiment analysis and confidence scores
df['sentiment'] = None
df['confidence'] = None

# Loop through the DataFrame and run the sentiment analysis on each player bio (~40 minutes on a CPU)
for index, row in df.iterrows():
    result = nlp(row['Player Bio'])
    df.at[index, 'sentiment'] = result[0]['label']
    df.at[index, 'confidence'] = result[0]['score']


In [ ]:
# Create a new column for adjusted confidence
df['adjusted_confidence'] = None

# If the sentiment is negative, subtract the confidence score from 1
for index, row in df.iterrows():
    if row['sentiment'] == 'NEGATIVE':
        df.at[index, 'adjusted_confidence'] = 1 - row['confidence']
    else:
        df.at[index, 'adjusted_confidence'] = row['confidence']
